In [1]:
import geopandas

In [6]:
import geopandas as gpd
import pandas as pd




In [7]:
# Read the earthquake data file into a Pandas DataFrame
earthquake_data = pd.read_csv('earthquake_data.csv')

In [8]:
# Create a GeoDataFrame from the DataFrame with Point geometries
geometry = gpd.points_from_xy(earthquake_data.longitude, earthquake_data.latitude)
gdf = gpd.GeoDataFrame(earthquake_data, geometry=geometry)


In [9]:
print(gdf.columns)


Index(['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'nst',
       'gap', 'dmin', 'rms', 'net', 'id', 'updated', 'place', 'type',
       'horizontalError', 'depthError', 'magError', 'magNst', 'status',
       'locationSource', 'magSource', 'geometry'],
      dtype='object')


In [10]:
# Check data types
print(gdf.dtypes)

time                 object
latitude            float64
longitude           float64
depth               float64
mag                 float64
magType              object
nst                 float64
gap                 float64
dmin                float64
rms                 float64
net                  object
id                   object
updated              object
place                object
type                 object
horizontalError     float64
depthError          float64
magError            float64
magNst              float64
status               object
locationSource       object
magSource            object
geometry           geometry
dtype: object


In [11]:
# Check missing values
print(gdf.isnull().sum())

time                0
latitude            0
longitude           0
depth               0
mag                 0
magType             0
nst                17
gap                17
dmin               18
rms                 0
net                 0
id                  0
updated             0
place               2
type                0
horizontalError    17
depthError          1
magError           17
magNst             17
status              0
locationSource      0
magSource           0
geometry            0
dtype: int64


In [12]:
# Descriptive statistics
print(gdf.describe())

        latitude   longitude       depth        mag        nst         gap  \
count  26.000000   26.000000   26.000000  26.000000   9.000000    9.000000   
mean   52.101617 -140.218104   34.401538   2.779615  61.555556   58.288411   
std    13.825440   16.697624   42.131144   0.295736  22.661151   57.659757   
min    19.334999 -166.557700    0.000000   2.500000  22.000000   15.000000   
25%    38.767709 -151.066750    2.707500   2.537500  49.000000   25.000000   
50%    59.931850 -147.901100   12.150000   2.665000  64.000000   36.000000   
75%    61.299700 -122.802874   59.500000   2.992500  65.000000   61.595703   
max    69.593000  -98.035400  133.200000   3.400000  97.000000  188.000000   

           dmin        rms  horizontalError  depthError  magError     magNst  
count  8.000000  26.000000         9.000000   25.000000  9.000000   9.000000  
mean   0.059037   0.412820         0.457720    2.146000  0.414556  32.555556  
std    0.095741   0.264098         0.580343    7.417197  0.5